# Imports and Utilities

In [1]:
from typing import List, Tuple
import numpy as np
import pandas as pd
from IPython.display import Markdown
import optuna
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.model_selection import cross_val_score

COMPETITION_NAME = "march-machine-learning-mania-2023"
VERBOSE = False

pd.set_option("display.max_columns", None)

def show_df(df: pd.DataFrame, name: str = "DataFrame", verbose: bool = VERBOSE):
    if verbose:
        display(Markdown(f"## {name}"))
        display(Markdown("### Head"))
        display(df.head())
        display(Markdown("### Tail"))
        display(df.tail())
        display(Markdown("### Description"))
        display(df.describe())


def extract_data(filename: str, competition_name:str = COMPETITION_NAME) -> pd.DataFrame:
    mens_filepath = f"/kaggle/input/{competition_name}/M{filename}.csv"
    womens_filepath = f"/kaggle/input/{competition_name}/W{filename}.csv"
    try:
        df_mens = pd.read_csv(mens_filepath)
    except FileNotFoundError:
        df_mens = None
    try:
        df_womens = pd.read_csv(womens_filepath)
    except FileNotFoundError:
        df_womens = None
    df = pd.concat([df_mens, df_womens])
    show_df(df, filename)
    return df


def get_team_features(detailed_results: pd.DataFrame) -> pd.DataFrame:
    df = detailed_results.copy()
    df = clean_detailed_results(df)
    df = transform_game_to_team(df)
    df = transform_team_results(df)
    show_df(df)
    return df


def get_seed_features(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["Season"] > 2002
    df = df[mask]
    df["Seed"] = df["Seed"].str.replace(r"\D+","", regex=True)
    df["Seed"] = df["Seed"].astype(int)
    show_df(df)
    return df


def get_ranking_features(df_in:pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["RankingDayNum"] == df["RankingDayNum"].max()
    df = df[mask]
    df.drop(["SystemName", "RankingDayNum"], axis=1, inplace=True)
    df = df.groupby(["Season", "TeamID"]).agg("median")
    df = df.reset_index()
    show_df(df)
    return df


def get_game_outcomes(df):
    input_rows = df.to_records()
    output_rows = []
    for input_row in input_rows:
        output_rows.extend(parse_row(input_row))
    out_df = pd.DataFrame(output_rows)
    return out_df

def parse_row(row):
    season = row['Season']
    winning_team_id = row['WTeamID']
    losing_team_id = row['LTeamID']
    if winning_team_id < losing_team_id:
        small_id = winning_team_id
        big_id = losing_team_id
        outcome = True
    elif losing_team_id < winning_team_id:
        small_id = losing_team_id
        big_id = winning_team_id
        outcome = False
    records = [
        {
            "ID": f"{season}_{small_id}_{big_id}",
            'Season': season,
            'LowID': small_id,
            'HighID': big_id,
            'Win': outcome
        },
        {
            "ID": f"{season}_{big_id}_{small_id}",
            'Season': season,
            'LowID': big_id,
            'HighID': small_id,
            'Win': not outcome
        },
    ]
    return records


def clean_detailed_results(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(["WLoc", "DayNum"], axis=1)


def transform_game_to_team(game_results: pd.DataFrame) -> pd.DataFrame:
    winners = rename_columns(game_results, "W")
    loosers = rename_columns(game_results, "L")
    team_results = pd.concat((winners, loosers))
    team_results.drop(["TeamIDOpp"], axis=1, inplace=True)
    return team_results


def transform_team_results(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(["Season", "TeamID"]).median()
    df["FGP"] =  df["FGM"] / df["FGA"]
    df["FGP3"] =  df["FGM3"] / df["FGA3"]
    df["FTP"] =  df["FTM"] / df["FTA"]
    df["FGPOpp"] =  df["FGMOpp"] / df["FGAOpp"]
    df["FGP3Opp"] =  df["FGM3Opp"] / df["FGA3Opp"]
    df["FTPOpp"] =  df["FTMOpp"] / df["FTAOpp"]
    return df.reset_index()
    
    
def rename_columns(df: pd.DataFrame, team_prefix: str) -> pd.DataFrame:
    df = df.copy()
    df.columns =  (rename_column(column_name, team_prefix) for column_name in df.columns)
    return df


def rename_column(column_name: str, team_prefix: str) -> pd.DataFrame:
    if team_prefix == "W":
        opponent_prefix = "L"
    elif team_prefix == "L":
        opponent_prefix = "W"
    else:
        raise ValueError
    if column_name.startswith(team_prefix):
        column_name = column_name.lstrip(team_prefix)
    elif column_name.startswith(opponent_prefix):
        column_name = f"{column_name.lstrip(opponent_prefix)}Opp"
    return column_name


def split_winner_and_looser_columns(df: pd.DataFrame) -> Tuple[List[str], List[str]]:
    winner_columns = [name for name in df.columns if not name.startswith("L")]
    looser_columns = [name for name in df.columns if not name.startswith("W")]
    return winner_columns, looser_columns


def merge_features(
    season_features: pd.DataFrame, 
    tournament_features: pd.DataFrame, 
    seed_features: pd.DataFrame, 
    ranking_features: pd.DataFrame
) -> pd.DataFrame:
    features = pd.merge(
        season_features,
        tournament_features,
        how="inner",
        on=["Season", "TeamID"],
        suffixes=("Reg", "Tou")
    )
    features = features.merge(
        seed_features,
        how="inner",
        on=["Season", "TeamID"]
    )
    features = features.merge(
        ranking_features,
        how="left",
        on=["Season", "TeamID"],
    )
    show_df(features)
    return features


def merge_outcomes_with_features(outcomes: pd.DataFrame, features: pd.DataFrame, how: str = "inner") -> pd.DataFrame:
    feature_names = [name for name in features.columns if name not in ["Season", "TeamID", "Gender"]]
    data = pd.merge(
        outcomes, 
        features, 
        how=how, 
        left_on=["Season", "HighID"], 
        right_on=["Season", "TeamID"]
    )
    data = pd.merge(
        data, 
        features, 
        how=how, 
        left_on=["Season", "LowID"], 
        right_on=["Season", "TeamID"],
        suffixes=("High", "Low")
    )
    for name in feature_names:
        data[f"{name}Diff"] = data[f"{name}High"] - data[f"{name}Low"]
        data.drop([f"{name}High", f"{name}Low"], axis=1, inplace=True)
    data.drop(
        ["Season", "HighID", "LowID","TeamIDHigh","TeamIDLow"], 
        axis=1, 
        inplace=True
    )
    data.set_index("ID", inplace=True)
    show_df(data)
    return data


def get_submission_outcomes() -> pd.DataFrame:
    sample_submission = pd.read_csv(f"/kaggle/input/{COMPETITION_NAME}/SampleSubmission2023.csv")
    df = sample_submission.copy()
    df.drop("Pred", axis=1, inplace=True)
    df[["Season", "LowID", "HighID"]] = df["ID"].str.split("_", expand=True)
    df[["Season", "LowID", "HighID"]] = df[["Season", "LowID", "HighID"]].astype(int)
    show_df(df)
    return df


def objective(trial: optuna.Trial, X_train, y_train):
    params = {
        "objective": "binary",
        "metric": "l2",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    model = lgb.LGBMClassifier(**params)
    score = np.mean(cross_val_score(model, X_train, y_train, scoring="neg_brier_score", cv=5))
    return score


def run_study(X_train, y_train):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=100)
    return study


def train_optimized_model(study, X, y):
    params = study.best_params
    model = lgb.LGBMClassifier(
        objective="binary",
        metric="l2",
        verbosity=-1,
        boosting_type="gbdt",
        **params
    )
    model = model.fit(X, y)
    return model

# Load Data

In [2]:
season_results = extract_data("RegularSeasonDetailedResults")
tournament_results = extract_data("NCAATourneyDetailedResults")
seeds = extract_data("NCAATourneySeeds")
rankings = extract_data("MasseyOrdinals_thru_Season2023_Day128")

# Transform Data

In [3]:
season_team_features = get_team_features(season_results)

tournament_results["Season"] += 1
tournament_team_features = get_team_features(tournament_results)

seed_features = get_seed_features(seeds)

ranking_features = get_ranking_features(rankings)

## Merge features

In [4]:
features = merge_features(season_team_features, tournament_team_features, seed_features, ranking_features)

## Build Datasets

In [5]:
from sklearn.model_selection import train_test_split

data = extract_data("NCAATourneyCompactResults")
data_train, data_valid = train_test_split(data, random_state=0, test_size=0.1)

outcomes = get_game_outcomes(data)
outcomes_train = get_game_outcomes(data_train)
outcomes_valid = get_game_outcomes(data_valid)
outcomes_submission = get_submission_outcomes()

In [6]:
features_train = merge_outcomes_with_features(outcomes_train, features)
features_valid = merge_outcomes_with_features(outcomes_valid, features)
features_full = merge_outcomes_with_features(outcomes, features)

In [7]:
y_train = features_train["Win"]
X_train = features_train.drop("Win", axis=1)
y_valid = features_valid["Win"]
X_valid = features_valid.drop("Win", axis=1)
y = features_full["Win"]
X = features_full.drop("Win", axis=1)
X_submission = merge_outcomes_with_features(outcomes_submission, features, how="left").fillna(0)

# Step 4: Train a model


In [8]:
study = run_study(X_train, y_train)
study.best_params

[I 2023-03-15 17:17:19,111] A new study created in memory with name: no-name-d71d87f6-93a2-4e67-a856-c5ce27b54a6c


[LightGBM] [Warning] bagging_fraction is set=0.6913795388664588, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6913795388664588
[LightGBM] [Warning] lambda_l1 is set=1.1933746666297976e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1933746666297976e-07
[LightGBM] [Warning] feature_fraction is set=0.6842094302713124, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6842094302713124
[LightGBM] [Warning] lambda_l2 is set=0.04068307042421321, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.04068307042421321
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.6913795388664588, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6913795388664588
[LightGBM] [Warning] lambda_l1 is set=1.1933746666297976e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1933746666297976e-07
[LightGBM] [Warning] featu

[I 2023-03-15 17:17:19,761] Trial 0 finished with value: -0.22386461757226833 and parameters: {'lambda_l1': 1.1933746666297976e-07, 'lambda_l2': 0.04068307042421321, 'num_leaves': 55, 'feature_fraction': 0.6842094302713124, 'bagging_fraction': 0.6913795388664588, 'bagging_freq': 6, 'min_child_samples': 39}. Best is trial 0 with value: -0.22386461757226833.


[LightGBM] [Warning] bagging_fraction is set=0.6913795388664588, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6913795388664588
[LightGBM] [Warning] lambda_l1 is set=1.1933746666297976e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1933746666297976e-07
[LightGBM] [Warning] feature_fraction is set=0.6842094302713124, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6842094302713124
[LightGBM] [Warning] lambda_l2 is set=0.04068307042421321, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.04068307042421321
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] bagging_fraction is set=0.8836980185889741, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8836980185889741
[LightGBM] [Warning] lambda_l1 is set=0.001427074272378841, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.001427074272378841
[LightGBM] [Warning] feature_f

[I 2023-03-15 17:17:20,379] Trial 1 finished with value: -0.21127542921520215 and parameters: {'lambda_l1': 0.001427074272378841, 'lambda_l2': 0.0017886539532516142, 'num_leaves': 209, 'feature_fraction': 0.852619972117554, 'bagging_fraction': 0.8836980185889741, 'bagging_freq': 3, 'min_child_samples': 76}. Best is trial 1 with value: -0.21127542921520215.


...

[I 2023-03-15 17:17:57,193] Trial 98 finished with value: -0.1987366447184217 and parameters: {'lambda_l1': 3.084842593173705e-07, 'lambda_l2': 6.170566861236273, 'num_leaves': 236, 'feature_fraction': 0.4970805817048882, 'bagging_fraction': 0.4667731071643576, 'bagging_freq': 7, 'min_child_samples': 86}. Best is trial 89 with value: -0.19518578813741502.


[LightGBM] [Warning] bagging_fraction is set=0.4667731071643576, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4667731071643576
[LightGBM] [Warning] lambda_l1 is set=3.084842593173705e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.084842593173705e-07
[LightGBM] [Warning] feature_fraction is set=0.4970805817048882, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4970805817048882
[LightGBM] [Warning] lambda_l2 is set=6.170566861236273, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.170566861236273
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] bagging_fraction is set=0.4667731071643576, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4667731071643576
[LightGBM] [Warning] lambda_l1 is set=3.084842593173705e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.084842593173705e-07
[LightGBM] [Warning] feature_fract

[I 2023-03-15 17:17:57,869] Trial 99 finished with value: -0.20347408963666985 and parameters: {'lambda_l1': 5.337012670112608e-08, 'lambda_l2': 1.072244239701656, 'num_leaves': 247, 'feature_fraction': 0.4268505652670811, 'bagging_fraction': 0.447752383766269, 'bagging_freq': 7, 'min_child_samples': 83}. Best is trial 89 with value: -0.19518578813741502.


[LightGBM] [Warning] bagging_fraction is set=0.447752383766269, subsample=1.0 will be ignored. Current value: bagging_fraction=0.447752383766269
[LightGBM] [Warning] lambda_l1 is set=5.337012670112608e-08, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.337012670112608e-08
[LightGBM] [Warning] feature_fraction is set=0.4268505652670811, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4268505652670811
[LightGBM] [Warning] lambda_l2 is set=1.072244239701656, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.072244239701656
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


{'lambda_l1': 5.370123391392152e-07,
 'lambda_l2': 9.945353884750187,
 'num_leaves': 238,
 'feature_fraction': 0.5009761023678655,
 'bagging_fraction': 0.4100356630490645,
 'bagging_freq': 7,
 'min_child_samples': 89}

In [9]:
model = train_optimized_model(study, X_train, y_train)
preds = model.predict_proba(X_valid)[:,1]
print("Brier Score:", brier_score_loss(y_valid, preds))
print("ROC AUC:", roc_auc_score(y_valid, preds))
print("Parameters")
print(*(f"- {key}: {value}" for key, value in model.get_params(deep=True).items()), sep="\n")
print("Features")
print(*(f"- {name}: {imp}" for name, imp in sorted(zip(model.feature_name_, model.feature_importances_), key=lambda x: x[1], reverse=True)), sep="\n")


[LightGBM] [Warning] bagging_fraction is set=0.4100356630490645, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4100356630490645
[LightGBM] [Warning] lambda_l1 is set=5.370123391392152e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.370123391392152e-07
[LightGBM] [Warning] feature_fraction is set=0.5009761023678655, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5009761023678655
[LightGBM] [Warning] lambda_l2 is set=9.945353884750187, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.945353884750187
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
Brier Score: 0.16191665593248014
ROC AUC: 0.8451003086419754
Parameters
- boosting_type: gbdt
- class_weight: None
- colsample_bytree: 1.0
- importance_type: split
- learning_rate: 0.1
- max_depth: -1
- min_child_samples: 89
- min_child_weight: 0.001
- min_split_gain: 0.0
- n_estimators: 100
- n_jobs: -1
- num_leaves: 

# Submit to the competition

We"ll begin by using the trained model to generate predictions, which we"ll save to a CSV file.

In [10]:
# Use the model to generate predictions
model.fit(X, y)
predictions = model.predict_proba(X_submission)[:,1]

# Save the predictions to a CSV file
output = pd.DataFrame({"ID": X_submission.index,
                       "Pred": predictions})
output.to_csv("submission.csv", index=False)
print(output.shape)
output.describe()

[LightGBM] [Warning] bagging_fraction is set=0.4100356630490645, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4100356630490645
[LightGBM] [Warning] lambda_l1 is set=5.370123391392152e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.370123391392152e-07
[LightGBM] [Warning] feature_fraction is set=0.5009761023678655, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5009761023678655
[LightGBM] [Warning] lambda_l2 is set=9.945353884750187, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.945353884750187
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
(130683, 2)


,Pred
count,130683.000000
mean,0.466208
std,0.020941
min,0.067112
25%,0.466034
50%,0.466034
75%,0.466034
max,0.909202
